In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 14.6 MB/s 
     |████████████████████████████████| 7.6 MB 48.6 MB/s 
     |████████████████████████████████| 182 kB 53.2 MB/s 


In [2]:
import transformers

In [3]:
from transformers import ElectraTokenizer

In [5]:
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")
tokenizer.tokenize("[CLS] 한국어 ELECTRA를 공유합니다. [SEP]")

Downloading:   0%|          | 0.00/263k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/61.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/467 [00:00<?, ?B/s]

['[CLS]', '한국어', 'EL', '##EC', '##TRA', '##를', '공유', '##합니다', '.', '[SEP]']

In [6]:
tokenizer('안녕하세요')

{'input_ids': [2, 11655, 4279, 8553, 3], 'token_type_ids': [0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1]}

In [10]:
tokenizer.tokenize("뚫딻띓. 안녕 오늘은 날씨가 참 좋네요")

['[UNK]', '.', '안녕', '오늘', '##은', '날씨', '##가', '참', '좋', '##네', '##요']

토크나이저 훈련

1. 어떤 기법을 사용할지
2. vocab_size 결정 >> 작으면 메모리 이득, 크면 의미를 잘 표현할 수 있다
3. text corpus >> 통계적으로 분석, vocab

tokenizer에 vocab 수동으로 직접 추가

vocab 300 ~ 350 [UNK1], [UNK1], [UNK1] ...
고유명사 언노운의 더미를 vocab에 추가하면 된다.



In [ ]:
#tokenizer.vocab

tokenizer.vocab

In [11]:
# 숫자로 변환
tokenizer.convert_tokens_to_ids(['[UNK]', '.', '안녕', '오늘', '##은', '날씨', '##가', '참', '좋', '##네', '##요'])

[1, 18, 11655, 6451, 4112, 8413, 4070, 3425, 3311, 4116, 4150]

In [12]:
tokenizer.convert_ids_to_tokens([0,1,2,3, 155,156, 30000,30001, 350001])

['[PAD]', '[UNK]', '[CLS]', '[SEP]', '※', '‼', '킬로그램', '태백산', '[UNK]']

In [14]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [15]:
tokenizer

PreTrainedTokenizer(name_or_path='bert-base-uncased', vocab_size=30522, model_max_len=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

특수한 토큰들
unk, sep, pad, cls, mask
사용법은 나중에 알아보는걸로

In [16]:
tokenizer.tokenize('안녕하세요')

['ᄋ',
 '##ᅡ',
 '##ᆫ',
 '##ᄂ',
 '##ᅧ',
 '##ᆼ',
 '##ᄒ',
 '##ᅡ',
 '##ᄉ',
 '##ᅦ',
 '##ᄋ',
 '##ᅭ']

영어(알파벳)을 기반으로 만든 모델이기 때문에 위와 같은 결과가 나온다

In [17]:
tokenizer.tokenize('hello, world!')

['hello', ',', 'world', '!']

In [18]:
tokenizer.vocab_size

30522

In [23]:
# 한국어 지원이 가능한 토크나이저 사용

multi_tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")

Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

In [24]:
multi_tokenizer.tokenize('안녕하세요')

['안', '##녕', '##하', '##세', '##요']

multi : '안녕' << vocab에 포함 X

electra : '안녕' 이 vocab에 포함

의미부여가 가능해짐

In [25]:
multi_tokenizer.vocab_size

119547

In [26]:
# BPE 토크나이저 실습
from transformers import RobertaTokenizer

tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [27]:
tokenizer.convert_ids_to_tokens([111,22,33])

['Ġ-', 'Ġ"', 'Ġhave']

BPE의 경우 두개의 토큰 조합으로 모든 단어를 표현할수있다. 

언어 모델

텍스트 >> 모델 >> 훈련

BERT의 경우
긁어온 문장에서 MASK를 이용해서 가리고(Input)

들어가야할 단어를 맞추는 식(Output)의 훈련 반복

**3가지 종류 언어모델**

>> 언어에 대한 통계적 이해를 가진 모델

1. Decoder-only 모델 (GPT1, GPT2, GPT3, ..
   - few shot learning 사용
   - 단점: 모델이 너무 큼
   - Auto regressive model이라고도 함
   - 점점 더 모델이 커지고 있음(모델이 크고, 데이터를 많이 넣을수록 성능이 좋다)



2. Encoder-only 모델 (BERT, RoBERTa, Electra, Albert, ...
  - input을 vector 로 바꿔준다.
  - vector는 의미와 관련있다.
  - Auto encoder model 이라고도 함


   - Pre-training(사전학습, 비지도학습) >> fine-tuning(지도학습)
   - Downstream task(번역, 문장 분류, 질의응답, ...)
   - Why? 
      1. 성능이 기존 방식보다 훨씬 좋다
      2. 소량의 데이터로도 학습이 훨씬 빨리된다



3. Encoder-Decoder 모델 >> Transformer 모델과 구조는 동일(BART, T5)

  - Pre-training(사전학습, 비지도학습) >> fine-tuning(지도학습)
  - 생성관련 task에 사용



Encoder 모델은 생성 task 불가능, 추출은 가능
Decoder 모델은 생성 taks 가능


Ex) 챗봇을 만들고 싶다.
시나리오 1. Encoder모델로 챗봇을 만들려면?
>> (답변1, 답변2, 답변3,...)
질문 > 모범답안을 뽑아낸다.

시나리오 2. Decoder모델로 챗봇을 만들면?
>> 성차별적인 발언, 인종차별적인 발언을 할 가능성이 있음

Embedding?
토큰을 단어로 바꿔줌

고양이, 낙타, 의자

의미적 유사도
고양이-낙타 1
낙타-의자 2

당연히 1번 동물이라는 범주에 속해있다.

고양이 -->1-->vector[1,1]
낙타 --> 241 --> vector[0.5,2.5]
의자 --> 11245 --> vector[-1.8, -7]

1   1

05   2.5

-1.8   -7

Embedding >> 훈련 O
Encoding >> 훈련 X

Embedding을 통해 단어(글자)형태을 vector화
Encoding을 통해 vector에 의미를 부여하기위한 vector화

vector화 처리가 된 데이터를 훈련하기위한 목표 언어로 바꾸는 과정이 Decoders 과정



구글 NLP 논문
Attention is All you Need

Attention


i Q(1,2,3) K(1,3,0) V(1,2,3)

am Q(4,0,3) K(1,1,0) V(1,0,3)

a Q(1,2,3) K(1,3,0) V(1,2,3)

student Q(1,2,3) K(1,3,0) V(1,2,3)

Q 벡터와 K 벡터를 내적한뒤 V를 곱하고 더함
위 계산 과정을 통해 각 토큰에서 의미연관을 위한 가중치를 더하게 된다.
여기서 가중치는 Q벡터 K벡터 내적을 통해 계산가능

정리하자면
Decoder에서 추론 시점에 Encoder 토큰을 얼마나 참조하는지 key query에서 가중치

Self-attention
Encoder가 되기전 Embedding 과정에서 attention 과정을 통해 별도의 추가 과정을 가지지않도록 하는 방법

2017년 google Attention is All you Need, transformer NLP가 급격히 성자하기 시작했다.



2019년 BERT 모델 >> transformer에서 Encoder만 뗀 모델


문장 전체의 의미를 나타내는 벡터가 뭐지?
>> 각 토큰이 나타내는 벡터의 평균


질의응답

[CLS] 포메라니안은 어떤걸 좋아해?
[SEP] 포메라니안은 독일 태생의 사녕견으로, 노는걸 좋아한다.


nlp
훈련
일반적으로 배치 사이즈 클수록 좋음



한국어 Pretrained model 설명

오픈소스중 추천 모델

 - koelectra
 - klue-roberta
 - koBERT

초대형 언어모델
gpt3


AWS SAM (서버리스 배포) : 작은모델 이용 적합
AWS EC2 : 비싸




